<a href="https://colab.research.google.com/github/KiritiVelivela/Asaan/blob/master/EVA4_B2_A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, bias=False) #Input 28x28x1, Output 26x26x8, RF 3,
        self.bnom1 = nn.BatchNorm2d(8)
        self.drop1 = nn.Dropout(0.05) #Probability 5%  

        self.conv2 = nn.Conv2d(8, 8, 3, bias=False) #Input 26x26x8, Output 24x24x8, RF 5
        self.bnom2 = nn.BatchNorm2d(8)
        self.drop2 = nn.Dropout(0.05) #Probability 5% 

        self.conv3 = nn.Conv2d(8, 8, 3, bias=False) #Input 24x24x8, Output 22x22x8, RF 7
        self.bnom3 = nn.BatchNorm2d(8)
        self.drop3 = nn.Dropout(0.05) #Probability 5% 

        self.conv4 = nn.Conv2d(8, 8, 3, bias=False) #Input 22x22x8, Output 20x20x8, RF 9
        self.bnom4 = nn.BatchNorm2d(8)
        self.drop4 = nn.Dropout(0.05) #Probability 5% 

        self.pool1 = nn.MaxPool2d(2, 2) # Input 20x20x8 Output 10x10x8 RF 10

        self.conv5 = nn.Conv2d(8, 16, 3, bias=False) #Input 10x10x8, Output 8x8x16, RF 14
        self.bnom5 = nn.BatchNorm2d(16)
        self.drop5 = nn.Dropout(0.05) #Probability 5%  
        
        self.conv6 = nn.Conv2d(16, 16, 3, bias=False) #Input 8x8x16, Output 6x6x16, RF 18
        self.bnom6 = nn.BatchNorm2d(16)
        self.drop6 = nn.Dropout(0.05) #Probability 5%  

        self.conv7 = nn.Conv2d(16, 16, 3, bias=False) #Input 6x6x16, Output 4x4x16, RF 22
        self.bnom7 = nn.BatchNorm2d(16)
        self.drop7 = nn.Dropout(0.05) #Probability 5%  

        self.conv8 = nn.Conv2d(16, 16, 3, bias=False) #Input 4x4x16, Output 2x2x16, RF 26
        self.bnom8 = nn.BatchNorm2d(16)
        self.drop8 = nn.Dropout(0.05) #Probability 5%  
        
        self.conv9 = nn.Conv2d(16, 10, 1, bias=False) #Input 2x2x16, Output 2x2x10, RF 26

    def forward(self, x):
        x = F.relu(self.bnom1(self.drop1(self.conv1(x))))
        x = F.relu(self.bnom2(self.drop2(self.conv2(x))))
        x = F.relu(self.bnom3(self.drop3(self.conv3(x))))
        x = F.relu(self.bnom4(self.drop4(self.conv4(x))))

        x = self.pool1(x)

        x = F.relu(self.bnom5(self.drop5(self.conv5(x))))
        x = F.relu(self.bnom6(self.drop6(self.conv6(x))))
        x = F.relu(self.bnom7(self.drop7(self.conv7(x))))
        x = F.relu(self.bnom8(self.drop8(self.conv8(x))))

        x = self.conv9(x)
        
        x = F.avg_pool2d(x, 2)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [47]:

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
           Dropout-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4            [-1, 8, 24, 24]             576
           Dropout-5            [-1, 8, 24, 24]               0
       BatchNorm2d-6            [-1, 8, 24, 24]              16
            Conv2d-7            [-1, 8, 22, 22]             576
           Dropout-8            [-1, 8, 22, 22]               0
       BatchNorm2d-9            [-1, 8, 22, 22]              16
           Conv2d-10            [-1, 8, 20, 20]             576
          Dropout-11            [-1, 8, 20, 20]               0
      BatchNorm2d-12            [-1, 8, 20, 20]              16
        MaxPool2d-13            [-1, 8, 10, 10]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [53]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print(" EPOCH -- ", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

 EPOCH --  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0691, Accuracy: 9823/10000 (98.23%)

 EPOCH --  2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0231, Accuracy: 9932/10000 (99.32%)

 EPOCH --  18


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0197, Accuracy: 9942/10000 (99.42%)

 EPOCH --  19


loss=0.07604975998401642 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.43it/s]



Test set: Average loss: 0.0214, Accuracy: 9938/10000 (99.38%)

